### Manipulation of PEAKS de novo results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_July2020_QE_T0/DN50_10lgP15/109_TROCAS_QE_July2020_DENOVO_18/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_July2020_QE_T0/DN50_10lgP15/109_TROCAS_QE_July2020_DENOVO_18


In [3]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks110 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_July2020_QE_T0/DN50_10lgP15/109_TROCAS_QE_July2020_DENOVO_18/109_TROCAS_QE_July2020_DENOVO_18_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks110))

print(peaks110.columns)

# These columns mess things up- get rid of them

del peaks110['Fraction']
del peaks110['Scan']
del peaks110['Source File']
del peaks110['Tag Length']
del peaks110['PTM']
del peaks110['tag (>=0%)']
del peaks110['mode']
del peaks110['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks110.columns = columns

mean_len = peaks110['length'].mean()
print(mean_len)

# look at the dataframe
peaks110.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 415
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
11.674698795180722


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,LSSPATLNSR,98,10,523.2860,2,36.86,155000000.0,1044.5564,1.0
1,VTN(+.98)AELAWELR,97,11,651.8459,2,71.35,NaN,1301.6616,12.1
2,WNQ(+.98)QFLNAAK,97,10,610.7996,2,41.91,1270000.0,1219.5986,-11.4
3,LSSPAELNSR,97,10,537.2836,2,39.84,90400.0,1072.5513,1.3
4,LADVTLPR,96,8,442.7642,2,48.40,2450000.0,883.5127,1.4


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [4]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks110['A'] = peaks110['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks110['C'] = peaks110['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks110['D'] = peaks110['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks110['E'] = peaks110['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks110['F'] = peaks110['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks110['G'] = peaks110['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks110['H'] = peaks110['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks110 output, there will be no isoleucines (they're lumped in with leucines)
peaks110['I'] = peaks110['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks110['K'] = peaks110['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks110['L'] = peaks110['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks110['M'] = peaks110['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks110['N'] = peaks110['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks110['P'] = peaks110['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks110['Q'] = peaks110['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks110['R'] = peaks110['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks110['S'] = peaks110['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks110['T'] = peaks110['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks110['V'] = peaks110['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks110['W'] = peaks110['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks110['Y'] = peaks110['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks110['c-carb'] = peaks110['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks110['m-oxid'] = peaks110['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks110['n-deam'] = peaks110['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks110['q-deam'] = peaks110['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks110['stripped_peptide'] = peaks110['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks110['stripped_length'] = peaks110['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks110['ptm-total'] = peaks110['c-carb'] + peaks110['m-oxid'] + peaks110['n-deam'] + peaks110['q-deam']

# calculate NAAF numerator for each peptide k
peaks110['NAAF_num.'] = peaks110['Area'] / peaks110['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks110.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/110_MS_T00_GF_DN/110B_MS_T00_GF_DN50.csv")

# check out the results
peaks110.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,LSSPATLNSR,98,10,523.2860,2,36.86,155000000.0,1044.5564,1.0,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,15500000.0
1,VTN(+.98)AELAWELR,97,11,651.8459,2,71.35,NaN,1301.6616,12.1,2,...,1,0,0,0,1,0,VTNAELAWELR,11,1,NaN
2,WNQ(+.98)QFLNAAK,97,10,610.7996,2,41.91,1270000.0,1219.5986,-11.4,2,...,1,0,0,0,0,1,WNQQFLNAAK,10,1,127000.0
3,LSSPAELNSR,97,10,537.2836,2,39.84,90400.0,1072.5513,1.3,1,...,0,0,0,0,0,0,LSSPAELNSR,10,0,9040.0
4,LADVTLPR,96,8,442.7642,2,48.40,2450000.0,883.5127,1.4,1,...,0,0,0,0,0,0,LADVTLPR,8,0,306250.0


In [5]:
# keep only stripped peptide I/L and NAAF
dn_110 = peaks110[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_110.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_110.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/110B_MS_T00_GF_DN50_ILnaafs.csv")

dn_110.head()

,stripped_peptide,Area,NAAF_num.
0,LSSPATLNSR,155000000.0,15500000.0
1,VTNAELAWELR,NaN,NaN
2,WNQQFLNAAK,1270000.0,127000.0
3,LSSPAELNSR,90400.0,9040.0
4,LADVTLPR,2450000.0,306250.0


In [6]:
# made a new dataframe that contains the sums of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks110['A'].sum(),
        'C': peaks110['C'].sum(),
        'D': peaks110['D'].sum(),
        'E': peaks110['E'].sum(),
        'F': peaks110['F'].sum(),
        'G': peaks110['G'].sum(),
        'H': peaks110['H'].sum(),
        'I': peaks110['I'].sum(),
        'K': peaks110['K'].sum(),
        'L': peaks110['L'].sum(),
        'M': peaks110['M'].sum(),
        'N': peaks110['N'].sum(),
        'P': peaks110['P'].sum(),
        'Q': peaks110['Q'].sum(),
        'R': peaks110['R'].sum(),
        'S': peaks110['S'].sum(),
        'T': peaks110['T'].sum(),
        'V': peaks110['V'].sum(),
        'W': peaks110['W'].sum(),
        'Y': peaks110['Y'].sum(),
        'c-carb': peaks110['c-carb'].sum(),
        'm-oxid': peaks110['m-oxid'].sum(),
        'n-deam': peaks110['n-deam'].sum(),
        'q-deam': peaks110['q-deam'].sum(),
        'Total area': peaks110['Area'].sum(),
        'Total length': peaks110['stripped_length'].sum()
       }

totalpeaks110 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks110['% C w/ carb'] = totalpeaks110['c-carb'] / totalpeaks110['C'] 

# calculate percentage of M's that are oxidized
totalpeaks110['% M w/ oxid'] = totalpeaks110['m-oxid'] / totalpeaks110['M'] 

# calculate percentage of N's that are deamidated
totalpeaks110['% N w/ deam'] = totalpeaks110['n-deam'] / totalpeaks110['N'] 

# calculate percentage of N's that are deamidated
totalpeaks110['% Q w/ deam'] = totalpeaks110['q-deam'] / totalpeaks110['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks110['NAAF denom.'] = totalpeaks110['Total area'] / totalpeaks110['Total length']

# write modified dataframe to new txt file
totalpeaks110.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/110_MS_T00_GF_DN/110B_MS_T00_GF_DN50_totals.csv")

totalpeaks110.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,452,143,138,233,94,224,176,0,300,602,...,59,71,6,7.301448e+08,4443,1.0,0.353293,0.23127,0.051724,164335.991328


In [7]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 164335.991328

# use NAAF >50% ALC to get NAAF factor
peaks110['NAAF factor'] = (peaks110['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksAAPTM_110 = peaks110[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_110['A-NAAF50'] = peaksAAPTM_110['A'] * peaks110['NAAF factor']
peaksAAPTM_110['C-NAAF50'] = peaksAAPTM_110['C'] * peaks110['NAAF factor']
peaksAAPTM_110['D-NAAF50'] = peaksAAPTM_110['D'] * peaks110['NAAF factor']
peaksAAPTM_110['E-NAAF50'] = peaksAAPTM_110['E'] * peaks110['NAAF factor']
peaksAAPTM_110['F-NAAF50'] = peaksAAPTM_110['F'] * peaks110['NAAF factor']
peaksAAPTM_110['G-NAAF50'] = peaksAAPTM_110['G'] * peaks110['NAAF factor']
peaksAAPTM_110['H-NAAF50'] = peaksAAPTM_110['H'] * peaks110['NAAF factor']
peaksAAPTM_110['I-NAAF50'] = peaksAAPTM_110['I'] * peaks110['NAAF factor']
peaksAAPTM_110['K-NAAF50'] = peaksAAPTM_110['K'] * peaks110['NAAF factor']
peaksAAPTM_110['L-NAAF50'] = peaksAAPTM_110['L'] * peaks110['NAAF factor']
peaksAAPTM_110['M-NAAF50'] = peaksAAPTM_110['M'] * peaks110['NAAF factor']
peaksAAPTM_110['N-NAAF50'] = peaksAAPTM_110['N'] * peaks110['NAAF factor']
peaksAAPTM_110['P-NAAF50'] = peaksAAPTM_110['P'] * peaks110['NAAF factor']
peaksAAPTM_110['Q-NAAF50'] = peaksAAPTM_110['Q'] * peaks110['NAAF factor']
peaksAAPTM_110['R-NAAF50'] = peaksAAPTM_110['R'] * peaks110['NAAF factor']
peaksAAPTM_110['S-NAAF50'] = peaksAAPTM_110['S'] * peaks110['NAAF factor']
peaksAAPTM_110['T-NAAF50'] = peaksAAPTM_110['T'] * peaks110['NAAF factor']
peaksAAPTM_110['V-NAAF50'] = peaksAAPTM_110['V'] * peaks110['NAAF factor']
peaksAAPTM_110['W-NAAF50'] = peaksAAPTM_110['W'] * peaks110['NAAF factor']
peaksAAPTM_110['Y-NAAF50'] = peaksAAPTM_110['Y'] * peaks110['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaksAAPTM_110['ccarb-NAAF50'] = peaksAAPTM_110['c-carb'] * peaksAAPTM_110['NAAF factor']
peaksAAPTM_110['moxid-NAAF50'] = peaksAAPTM_110['m-oxid'] * peaksAAPTM_110['NAAF factor']
peaksAAPTM_110['ndeam-NAAF50'] = peaksAAPTM_110['n-deam'] * peaksAAPTM_110['NAAF factor']
peaksAAPTM_110['qdeam-NAAF50'] = peaksAAPTM_110['q-deam'] * peaksAAPTM_110['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_110.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/110_MS_T00_GF_DN/110B_MS_T00_GF_DN50_naaf.csv")

peaksAAPTM_110.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,LSSPATLNSR,94.318961,1,0,0,0,0,0,0,0,...,94.318961,282.956884,94.318961,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
1,VTNAELAWELR,NaN,2,0,0,2,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WNQQFLNAAK,0.772807,2,0,0,0,1,0,0,1,...,0.000000,0.000000,0.000000,0.00000,0.772807,0.0,0.0,0.0,0.0,0.772807
3,LSSPAELNSR,0.055009,1,0,0,1,0,0,0,0,...,0.055009,0.165028,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.000000
4,LADVTLPR,1.863560,1,0,1,0,0,0,0,0,...,1.863560,0.000000,1.863560,1.86356,0.000000,0.0,0.0,0.0,0.0,0.000000


In [8]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMs

index = ['sample total']

data = {'NAAF': peaksAAPTM_110['NAAF factor'].sum(),
        'A': peaksAAPTM_110['A-NAAF50'].sum(),
        'C': peaksAAPTM_110['C-NAAF50'].sum(),
        'D': peaksAAPTM_110['D-NAAF50'].sum(),
        'E': peaksAAPTM_110['E-NAAF50'].sum(),
        'F': peaksAAPTM_110['F-NAAF50'].sum(),
        'G': peaksAAPTM_110['G-NAAF50'].sum(),
        'H': peaksAAPTM_110['H-NAAF50'].sum(),
        'I': peaksAAPTM_110['I-NAAF50'].sum(),
        'K': peaksAAPTM_110['K-NAAF50'].sum(),
        'L': peaksAAPTM_110['L-NAAF50'].sum(),
        'M': peaksAAPTM_110['M-NAAF50'].sum(),
        'N': peaksAAPTM_110['N-NAAF50'].sum(),
        'P': peaksAAPTM_110['P-NAAF50'].sum(),
        'Q': peaksAAPTM_110['Q-NAAF50'].sum(),
        'R': peaksAAPTM_110['R-NAAF50'].sum(),
        'S': peaksAAPTM_110['S-NAAF50'].sum(),
        'T': peaksAAPTM_110['T-NAAF50'].sum(),
        'V': peaksAAPTM_110['V-NAAF50'].sum(),
        'W': peaksAAPTM_110['W-NAAF50'].sum(),
        'Y': peaksAAPTM_110['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_110['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_110['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_110['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_110['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 164335.991328

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/110_MS_T00_GF_DN/110B_MS_T00_GF_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,538.867206,508.108632,189.51742,111.637515,88.591018,40.716572,136.907931,64.941118,0.0,54.060458,...,40.145725,189.51742,92.045954,34.185792,1.772419,1.0,0.744057,0.144867,0.091419,0.003279


## Export stripped peptides >50% ALC

In [9]:
##### keep only stripped peptide column 
pep50 = peaks110[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/110_MS_T00_GF_DN/110B_MS_T00_GF_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/110_MS_T00_GF_DN/110B_MS_T00_GF_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDN/110_MS_T00_GF_DN/110B_MS_T00_GF_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks110['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks110[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 415
# nonredundant stripped Peaks peptides >50% ALC 379
average peptide length Peaks peptides >50% ALC 10.706024096385542
# redundant Peaks peptides >50% ALC 415
# nonredundant Peaks peptides 386


,stripped_peptide
0,LSSPATLNSR
1,VTNAELAWELR
2,WNQQFLNAAK
3,LSSPAELNSR
4,LADVTLPR
